# Ozone

## Final renderings and combined plots

Examples for comparison

https://ozonewatch.gsfc.nasa.gov/facts/hole_SH.html

https://www.nasa.gov/feature/goddard/2019/2019-ozone-hole-is-the-smallest-on-record-since-its-discovery

In [ ]:
#SETTINGS
background = 'black' #'#555555'

#Earth model version tag
tag = '_flat'
#tag = '_flat_low' #Low res version
#(No need for high res topo for this visualisation)

size = (1920,1080) #Final render in 1080p

rotation = True

In [ ]:
try:
    import sys
    sys.path.insert(0, '../..') #for accessvis module
    import accessvis
    import numpy as np
    import datetime
    import math
    import pathlib
    import xarray as xr
    import matplotlib.pyplot as plt
    from matplotlib import cm
    import matplotlib
    import os
    import glob
    import json
    from scipy import interpolate
    from PIL import Image
    import matplotlib.dates as mdates
    from tqdm.notebook import tqdm
    import lavavu   
except (ImportError) as e:
    print('ImportError: Installing dependencies, please run this cell again...')
    !pip install xarray scipy h5netcdf dask netCDF4 pillow lavavu tqdm ipympl
    #warning: ipympl requires jupyter restart
    raise('Installed dependencies, please run this cell again')

# Get the data

Need access to project on gadi or local copy, edit datadir below if not on gadi

In [ ]:
#Assume running on gadi or with /g/data mounted, eg with sshfs
datadir = '/g/data/p73/archive/non-CMIP/CMORised/CCMI2022/CSIRO-ARCCSS/ACCESS-CM2-Chem/refD2/r1i1p1f1/Aday/toz/gn/v20220822/'
files = sorted(glob.glob(datadir + "*.nc"))

In [ ]:
for f in files:
    print(f)

In [ ]:
ds = xr.open_mfdataset(files, combine='nested', concat_dim="time")

In [ ]:
ds

### Load saved 3d model

Use a saved earth model and sphere model for the ozone plot

In [ ]:
#Create 3d viewer
lv = lavavu.Viewer(axis=False, border=0, verbose=0, background=background)

In [ ]:
#Main viewport with space to right for 2d plots
lv.viewport(0, 0, 0.65, 1.0, replace=True, title="-Main")

#cb = lv.colourbar(ticks=1, tickvalues=[220], format="%.5g DU", align='left', offset=40)

#lv["fontscale"] = 0.65

#Pre-saved topo data
with np.load('../../Earth/Topo' + tag + '.npz') as ldata:
    print(ldata)
    
    #Visualise
    shaders = ['triShader_test.vert', 'triShader_test.frag']
    #shaders = ['triShader_test.vert', 'triShader_grey.frag']
    for f in ['F', 'R', 'B', 'L', 'U', 'D']:
        verts = ldata[f]
        q = lv.quads(name=f, vertices=verts, texture='../../Earth/' + f + tag + '.png',
                     fliptexture=False, flip=f in ['F', 'L', 'D'], #Reverse facing
                     renderer="simpletriangles", opaque=True, shaders=shaders)
        print(q['texture'])
        q['saturation'] = 0.0 #[0,2]
        q['contrast'] = 0.8 #0.75  #[0,2]
        q['brightness'] = 0.0  #[-1,1]


In [ ]:
#Load the stratosphere mesh
#(requires running SphereMesh.ipynb if file not found)
with np.load('../../Earth/Sphere_6.4729.npz') as sdata:
    #tris0 = lv.objects['strato']
    lv.addstep(0) #Add a timestep or things don't work on load
    tris0 = lv.triangles("strato", **sdata) #vertices=sdata['vertices'], normals=sdata['normals'])
    
    #tris0['rotate'] = [0,-90,0] #This rotates the sphere to align with our [0,360] longitude texture
    tris0['texture'] = 'blank.png' #Need an initial texture or texcoords will not be generated
    tris0['renderer'] = 'sortedtriangles'
    lv['cullface'] = False #Must disable this for the ozone plot
    tris0["rotate"] = [0,0,0]
    #tris0["alpha"] = 1.0 #0.6
    
    if rotation:
        #Try some different shaders with the 3d rotating model
        tris0['shaders'] = ['triShader_oz.vert', 'triShader_oz2.frag']
    else:
        tris0['shaders'] = ['triShader_oz.vert', 'triShader_oz.frag']
    
    #WITH TITLE
    cb = tris0.colourbar(ticks=1, tickvalues=[220], format="%.5g DU", align='left', offset=16, size=[0.4,16], position=-40, outline=0) #For 640x480 render
    #NO TITLE
    #cb = tris0.colourbar(ticks=1, tickvalues=[220], format="%.5g DU", align='left', offset=16, size=[0.4,16], position=-16, outline=0)


In [ ]:
#2nd viewport for 2d plots to right
lv.viewport(0.65, 0, 0.35, 1.0, title="Ozone hole area (millions of square km)")

overlay = None

def render_with_overlay(overlay_image, size=[1.0, 1.0], offset=[0.0, 0.0]):
    global overlay
    if not overlay:
        #Create the 2d overlay plot - problems updating texture here unless this is the last object added
        overlay = lv.screen(shaders=['screen.vert', 'screen.frag'], vertices=[[0,0,0]],
                            uniforms={"size" : size, "offset" : offset})

    #Render the main 3d image
    overlay.texture(overlay_image)

    #Plot offset in full screen
    #t = lv['title']
    #lv['title'] = ''
    overlay['uniforms'] = {"size" : size, "offset" : offset}
    lv.render()
    #lv['title'] = t

#render_with_overlay('blank.png', size=[0.3, 0.3])
render_with_overlay('blank.png')

In [ ]:

#Lighting
lv['ambient'] = 0.3 #0.4
lv['diffuse'] = 1.0 #0.65

#Camera
#lv.resize(size[0] // 2, size[1] // 2)
lv.view(0) #Select first viewport
lv.resize(size[0], size[1])
lv.translation(0.157, -0.359, -17.961)
lv.rotation(-90.0, 0.0, 0.0)


In [ ]:
#lv.interactive() #Show window (linux/windows hosts only)
lv.window()

In [ ]:
lat = np.array(ds['lat'])
lon = np.array(ds['lon'])
times = np.array(ds['time'])
startdate = datetime.datetime.strptime('1960-01-01', "%Y-%m-%d").date()

In [ ]:
############################# CALIBRATION - LONGITUDE TEXTURE
#Create a calibration image to show longitude mapping of texture matches expectations

#Get todays timestep data to work on
df = ds.sel(time=datetime.datetime.now(), method='nearest')

#We can change the image to [-180,180] as follows,
#but it is very slow to run this on all the full dataset
#instead we will use the rotation property to shift the texture mapping for [0,360]
#a calibration image will show if this is working correctly
df.coords['lon'] = (df.coords['lon'] + 180) % 360 - 180
df = df.sortby(df.lon)
#df

In [ ]:
latb = np.array(df['lat_bnds'][:,0])
lonb = np.array(df['lon_bnds'][:,0])

lgrid = np.array(np.meshgrid(lonb, latb))
lgrid.shape

#sumgrid = lgrid[0] + lgrid[1]
#sumgrid.shape
longrid = lgrid[0]

mmin = longrid.min()
mmax = longrid.max()
print(mmin, mmax)

fig = plt.figure(frameon=False)
fig.set_size_inches(4,3)
#To make the content fill the whole figure
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
ax.imshow(longrid, aspect='auto')
#fig.savefig('longitude.png')

toz = longrid #########################
#Normalise
image = (toz-mmin)/(mmax-mmin)
#Clip out of [0,1] range - in case defined range is not the global minima/maxima
image = np.clip(image, 0, 1)
#print(image.shape)

#Apply colourmap
#tex = cm.viridis(image)
tex = cm.magma(image)
#print(tex)

#print("Range of timestep:", toz.min(),toz.max())

#print(tex.shape)
#Add alpha channel
tex_a = np.dstack((tex[::,::,0:3], image))

#Show the calibration plot on the globe model
#tris0.texture(tex_a)
#lv.translation(-3.241, -0.225, -17.858)
#lv.rotation(-107.232, -3.084, 166.483)
#lv.interactive()
##########################################################################

In [ ]:
#COLOURMAP SETUP
lv1 = lavavu.Viewer(border=False, axis=False, background="gray90", quality=3, fontscale=2)
o = lv1.colourbar()

cm0 = o.colourmap('twilight', reverse=True)
lv1.display(resolution=[640,50], transparent=True)

colours = [c[1] for c in cm0.tolist()]

#Remove first 32 colours to strip white from low end
colours = colours[32:]
print(len(colours))

colours2 = []

DIV=len(colours)//3
print(DIV)
#TWEAKS COLOURMAP TO FIX 220DU AS DIVERGING POINT
for i,c in enumerate(colours):
    #Skip every 2nd for first 1/3 of range
    if i < DIV and i % 2 == 0:
        continue
    colours2.append(c)

for i,c in enumerate(colours2):
    if i < 148:
        c[0] = min(int(c[0]*1.5), 255)

colours = colours2

lv1 = lavavu.Viewer(border=False, axis=False, background="gray90", quality=3, fontscale=2)
o = lv1.colourbar(ticks=1, tickvalues=[220], format="%.5g DU", align='left', offset=16, size=[0.9,16], outline=0) #, fontcolour='white')
cm0 = o.colourmap(colours, range=[0, 600])
lv1.display(resolution=[160,250], transparent=True)

In [ ]:
#Just use a fixed range, convert DU to M
mmin = 0 * 1e-5
mmax = 600 * 1e-5
#Setup centre value to be 220DU
#mmin = 0
#mmax = 440 * 1e-5

#Threshold : 220 DU
#Dobson Units : 1m = 1e5 DU
#Our scale is in metres
threshold = 220 * 1e-5
print("Threshold of 220 DU in M: ",threshold)
print("Range of data: ",mmin,mmax)

#Show colourbar with threshold marked in DUs
#cmap = lv.colourmap('viridis', range=(mmin * 1e5, mmax * 1e5))
#cmap = lv.colourmap('magma', range=(mmin * 1e5, mmax * 1e5))
clist = []
for i,c in enumerate(colours):
    pos = i/(len(colours)-1)
    clist.append((pos, c))

cmapname = None
#cmapname = 'hsv_r'

if cmapname:
    cmap = tris0.colourmap(cmapname, range=(mmin * 1e5, mmax * 1e5))
else:
    cmap = tris0.colourmap(clist, range=(mmin * 1e5, mmax * 1e5))


plot_inc=1 #10
R = [0,0,0]
if cmapname:
    mcm = getattr(cm, cmapname)
else:
    mcm = matplotlib.colors.LinearSegmentedColormap.from_list('custom', np.array(colours)[::,0:3] / 255.0) #, N=len(colours))

#print(mcm)
def get_frame_3d(t):
    global tex
    #for g in globe_objs:
    #    lv.objects[g].rotation = [0, 0.01*t, 0]

    day = startdate + datetime.timedelta(days=t)
    #lv['title'] = str(time[count]) + " " + day.strftime("%d %B, %Y")
    lv.view(0) #Select first viewport
    #lv['title'] = "" #"-Total column ozone and hole region (< 220 DU) " + day.strftime("%d %B, %Y").rjust(20)
    lv['title'] = "-Total column ozone : " + day.strftime("%B %Y")

    #Rotate the earth about it's axis
    if rotation:
        #Rotate around X to show both hemispheres throughout year
        #Need to show southern in latter half so the hole is visible
        #tt = day.timetuple()
        #yday = tt.tm_yday #Day of year
        #R[0] = (1.0 - math.sin((yday / 365.) * math.pi)) * 90
        #print("yday", yday, R[0])
        #Y axis rotation
        R[1] += 0.5 #0.1
        for o in ['F', 'R', 'B', 'L', 'U', 'D', 'strato']:
            obj = lv.objects[o]
            obj["rotate"] = R

    #Get timestep
    toz = np.array(ds['toz'][t])
    
    #print(toz.shape)
    #Normalise
    image = (toz-mmin)/(mmax-mmin)
    #Clip out of [0,1] range - in case defined range is not the global minima/maxima
    image = np.clip(image, 0, 1)

    #Apply colourmap
    #tex = cm.viridis(image)
    #tex = cm.magma(image)
    tex = mcm(image)
    #norm = matplotlib.colors.LogNorm()
    #i2 = norm(image)
    #tex = mcm(i2)
    #print(tex)

    #print("Range of timestep:", toz.min(),toz.max())

    #print(tex.shape)
    #Add alpha channel - passes the original normalised toz data
    tex_a = np.dstack((tex[::,::,0:3], image))

    tris0.texture(tex_a)

#lv.translation(0.37226, -0.897176, -15.9221)
#lv.rotation(0.0788629, 0.80106, -0.58636, -0.0913035)
#lv.translation(0.53, -1.145, -16.104)
lv.view(0)
#Angled for rotation view
if rotation:
    #lv.translation(-3.241, -0.225, -17.858)
    #lv.translation(0.288, -0.177, -17.5) #WITH TITLE
    lv.translation(0.0, 0.0, -17.087)     #NO TITLE
    lv.rotation(-107.232, -3.084, 166.483)
else:
    # Polar view, no rotating animation
    #lv.translation(-3.241, -0.225, -17.858)
    #lv.rotation(-80, 2, 45)
    #lv.translation(0, 0, -17.8)
    lv.translation(0.288, -0.177, -17.5)
    #lv.rotation(-90, 0, 125)

get_frame_3d(0)
#lv.interactive()

In [ ]:
lv.camera()

In [ ]:
#get_frame_3d(1000)
#24th October 2018
#get_frame_3d(21481)
#29th October 2017
get_frame_3d(21121)

In [ ]:
#lv.resize(size[0], size[1])
#lv.blend('pre')
#lv['contrast'] = None #1.25
#lv['brightness'] = None #-0.25
#lv['saturation'] = None #0.25

tris0['contrast'] = 1.2 #1.
tris0['brightness'] = 0.
tris0['saturation'] = 1.

In [ ]:
#img_3d = lv.rawimage(resolution=(640,480), channels=3)
#print(img_3d.data.shape)
lv.display(resolution=size)

In [ ]:
#Save last frame texture as image for testing
'''
print(tex.shape,tex.dtype)
print((tex * 255.0).astype(np.uint8))
from PIL import Image
im = Image.fromarray((tex * 255.0).astype(np.uint8))
im.save("oz.png")
''';

### Load saved data from Ozone-Calc

In [ ]:
dates = np.array(ds['time'])
years = dates.astype('datetime64[Y]').astype(int) + 1970
months = dates.astype('datetime64[M]').astype(int) % 12 + 1
days = (dates.astype('datetime64[D]') - dates.astype('datetime64[M]')).astype(int) + 1

cfn = "year_max.json"
afn = "all.json"
alldata = []
if os.path.exists(cfn):
    with open(cfn, "r") as infile:
        year_max = json.load(infile)
    with open(afn, "r") as infile:
        alldata = json.load(infile)
else:
    print('Please run Ozone-Calc.ipynb first')

In [ ]:
print(years[0], years[-1])

In [ ]:
#Get the axis data

#Years, sorted and duplicates removed
x = np.array(sorted(list(set(years))))
#Max hole area per year
y = np.array(list(year_max.values()))

#print(x, y)
#plt.plot(x,y)
#plt.show()

#create scatterplot to verify we have the data loaded
plt.scatter(x, y);

https://www.datatechnotes.com/2021/11/scattered-data-spline-fitting-example.html

In [ ]:
knot_numbers = 2 #Higher numbers fit the data closer, but look bad at the ends
x_new = np.linspace(0, 1, knot_numbers+2)[1:-1]
q_knots = np.quantile(x, x_new)

t,c,k = interpolate.splrep(x, y, t=q_knots, s=1)
yfit = interpolate.BSpline(t,c,k)(x)

In [ ]:
#3d and 2d plot combined
frame = np.zeros(dtype='uint8', shape=(size[1], size[0], 3))
print(frame.shape)

In [ ]:
# Enable interactive plot
#%matplotlib notebook
%matplotlib widget

In [ ]:
my_dpi = 200 #high_dpi work screen
#Fig size to fit two plots vertically in full res (size)
figsize = [7, 10.8] #int(size[1] / my_dpi), int(size[0] / my_dpi)]
print(figsize)
plt.rcParams['figure.figsize'] = figsize #[5.0, 8.0]
plt.rcParams['figure.dpi'] = my_dpi // 2
plt.rcParams.update({'font.size': 16})
#plt.rcParams["font.family"] = 'sans-serif' #"Arial"

In [ ]:
 #print(plt.style.available)
plt.style.use("dark_background")

# create a figure with axes, 2 subplots
#fig, (ax1, ax2) = plt.subplots(2, figsize=(5, 8), dpi=dpi)
fig, (ax1, ax2) = plt.subplots(2, figsize=figsize, dpi=my_dpi // 2)
fig.patch.set_facecolor(background)
ax1.set_facecolor(background)
ax2.set_facecolor(background)
canvas = fig.canvas

#plt.title("Ozone hole maximum size")
ax1.plot(x, y, '.', color="grey", label="original")
ax1.plot(x, yfit, '-', label="spline fit", color="mediumpurple")
#ax1.legend(loc='best', fancybox=True, shadow=True)
#ax1.grid()
ax1.set_xlabel('Year')
#ax1.set_ylabel('Area (million km²)')
ax1.set_ylabel('')
#plt.show()

# Hide the right and top spines
ax1.spines[['right', 'top']].set_visible(False)
#Remove axis margins
ax1.margins(x=0)
ax1.margins(y=0)

#Hide intermediate ticks
xlabels = ax1.xaxis.get_ticklabels()
#for xl in xlabels[1:-1]:
#    xl.set_visible(False)
#ylabels = ax1.yaxis.get_ticklabels()
#for yl in ylabels[0:-2]:
#    yl.set_visible(False)
ax1.set_ylim([0, 20])
ax1.set_yticks([0,20]) 

###### LINE WIDTHS ######
# change all spines
lw = 3 #2
for axis in ['bottom','left']:
    ax1.spines[axis].set_linewidth(lw)
    ax2.spines[axis].set_linewidth(lw)
# increase tick width
tw = lw #3 #2
ax1.tick_params(width=tw, length=10)
ax2.tick_params(width=tw, length=10)
########################

#ax = plt.gca()
# create a point in the axes
point, = ax1.plot(years[0],y[0], marker="o", color='seashell', markersize=10)
#point0, = ax1.plot(years[0],y[0], marker="o", color='white', markersize=5)
#bar, = ax.bar(1960,y[0], color='darkgrey')

######################## Months plot
ax2.set_xlabel('Month')
#ax2.set_ylabel('Area (million km²)')
ax2.set_ylabel('')

#Get Jan-Dec data
def get_months_plot(year):
    yl = list(years)
    i0 = yl.index(year)
    if year == years[-1]:
        i1 = len(dates)-1
    else:
        i1 = yl.index(year+1)
    return dates[i0:i1], alldata[i0:i1]

D = 0
date_list, yy = get_months_plot(years[D])
#print(D, date_list,yy)

curplot, = ax2.plot(date_list, yy, color="#c4bbff")
#Current day marker
point2, = ax2.plot(date_list[0],yy[0], marker="o", color='seashell', markersize=10)

# Set the locator
locator = mdates.MonthLocator()  # every month
# Specify the format - %b gives us Jan, Feb...
month_fmt = mdates.DateFormatter('%b')

X = plt.gca().xaxis
X.set_major_locator(locator)
# Specify formatter
X.set_major_formatter(month_fmt)

#Lock the ticks to Jan-Dec
#ticks = [np.datetime64('2000-%02d-01'%month) for month in range(1, 13)]
#ax2.set_xticks(ticks)
X.set_major_formatter(mdates.DateFormatter('%b'))

# Hide the right and top spines
ax2.spines[['right', 'top']].set_visible(False)
#Remove axis margins
ax2.margins(x=0.1)
#ax2.margins(y=0)
ax2.margins(y=0.25)

ax2.set_ylim([0, 20])
#Hide intermediate ticks in Y
#ylabels = ax2.yaxis.get_ticklabels()
#for yl in ylabels[0:-1]:
#    yl.set_visible(False)

ax2.set_yticks([0,20]) 

##########################################

outdir = 'frames/'
os.makedirs(outdir, exist_ok=True)
X = None
Y = -1

plt.show();

In [ ]:
def get_frame(idx):
    #Get 3d frame
    get_frame_3d(idx)

    #2d plot
    #Capture image data
    year = years[idx]

    #global X,Y   
    # set point's coordinates
    ts = year - 1960
    #point0.set_data([year],[y[ts]])
    #point.set_data([year],[yfit[ts]])
    point.set_data([year],[y[ts]])
    
    date_list, yy = get_months_plot(year)
    curplot.set_data(date_list,yy)
    ax2.set_xlim([date_list[0], date_list[-1]])
    #Current day marker
    point2.set_data([dates[idx]], [alldata[idx]])
    #print("POINT2",[dates[idx]], [alldata[idx]])

    canvas.draw()  # Draw the canvas, cache the renderer
    X = np.asarray(canvas.buffer_rgba())

    ## NEW METHOD, PLOT INSET WITH TEXTURE
    render_with_overlay(X)


In [ ]:
lv["fontscale"] = 0.7

In [ ]:
#29th October 2017
get_frame(21121)

In [ ]:
#lv.display((1920//2,1080//2))
lv.display((1920,1080))

In [ ]:
#raise(Exception())

In [ ]:
get_frame(0) #Reset to start

# Render animation

In [ ]:
frames = int(len(times))
print(frames)

#Faster animation - skip matplotlib animation functions
step = 1 #step increment in days
#step = 7
#step = 2 #Every 2nd day
start = 0 #Start from first

#Resume - find a date in the list
#start = list(dates).index(np.datetime64(datetime.datetime.strptime('Sep 30 2018  12:00PM', '%b %d %Y %I:%M%p')))

#Find a date in the list
single_year = False
#single_year = 2018
if single_year:
    ldates = list(dates)
    datestr = f'Jan 1 {single_year}  12:00PM'
    thedate = datetime.datetime.strptime(datestr, '%b %d %Y %I:%M%p')
    frame_idx = ldates.index(np.datetime64(thedate))
    start = frame_idx
    #Override frame count, just plot one year
    frames = start + 365

#for f in tqdm(range(start,frames,step), desc='Rendering loop'):
#    get_frame(f)
#fps=120 : 4x faster
#with lv.video(filename='ozone_v3.mp4', fps=120, resolution=(1920,1080), width=1920//2, height=1080//2, params="autoplay"):
with lv.video(filename='ozone_v6.mp4', fps=60, quality=3, resolution=(1920,1080), width=1920//2, height=1080//2, params="autoplay"):
    for f in tqdm(range(start,frames,step), desc='Rendering loop'):
        get_frame(f)

In [ ]:
#raise(Exception())

## Render single year plot

In [ ]:
year = 2019
step = 1 #step increment in days
#Find a date in the list
start = list(dates).index(np.datetime64(datetime.datetime.strptime(f'Jan 1 {year} 12:00PM', '%b %d %Y %I:%M%p')))
end = list(dates).index(np.datetime64(datetime.datetime.strptime(f'Dec 31 {year} 12:00PM', '%b %d %Y %I:%M%p')))
#Override frame count, just plot one year
frames = end - start + 1
print(frames)

with lv.video(filename=f'ozone_{year}.mp4', quality=3, resolution=(1920,1080), width=1920//2, height=1080//2, params="autoplay"):
    for f in tqdm(range(start,end,step), desc='Rendering loop'):
        get_frame(f)

## Render maximum extent for all years

In [ ]:
ylist = list(years)
def get_max_frame(y):
    i0 = ylist.index(y)
    if y == 2100:
        i1 = len(ylist)
    else:
        i1 = ylist.index(y+1)
    #Full year of data
    ydata = alldata[i0:i1]
    maxima = max(ydata)
    #print(dates[i0:i1])
    idx = ydata.index(maxima) + i0
    #print(maxima, idx)
    return idx

yrs = sorted(set(ylist))
lv.translation(0, 0, -17)
lv.rotation(-90.0, 0.0, 0.0)

rotation = False
with lv.video(filename=f'ozone_hole_maximums.mp4', quality=3, fps=3, resolution=(1920,1080), width=1920//2, height=1080//2, params="autoplay"):
    for yr in tqdm(yrs, desc='Rendering loop'):
        first = list(dates).index(np.datetime64(datetime.datetime.strptime(f'Jan 1 {yr} 12:00PM', '%b %d %Y %I:%M%p')))
        idx = get_max_frame(yr)
        #get_frame(idx+first)
        get_frame(idx)

In [ ]:
raise(Exception()) #Stop here

## Create a poster image

4K Res

In [ ]:
#Find a date in the list
get_frame_3d(list(dates).index(np.datetime64(datetime.datetime.strptime('Sep 21 2019  12:00PM', '%b %d %Y %I:%M%p'))))

In [ ]:
lv.viewport(0, 0, 1.0, 1.0) #, replace=True, title="")
lv.delete('screen1')

In [ ]:
lv.translation(0, 0, -17)
lv.rotation(-101.964, -22.039, 148.317)

lv.background('black')


In [ ]:
lv.hide('colourbar')
lv['title'] = ''
lv.display(resolution=(512,512))

In [ ]:
lv.image("poster_ozone_v2.png", resolution=(4096,4096), transparent=True)